In [ ]:
#要学会用histogram以及scalar来查看训练情况   主要是用于查看添加的神经网络层以及代价函数两部分的模块
#特别是用scalar来查看 cost_function是否有收敛，以此来检验所构造的机器学习模型的效果

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

#添加神经网络层
def add_layer(inputs,in_size,out_size,n_layer,activation_function=None):  #在这里增加了神经层的名字
    layer_name = 'layer%s' % n_layer   #这一行的作用就是
    with tf.name_scope('layer'):
        with tf.name_scope('weights'):
            Weights = tf.Variable(tf.random_normal([in_size,out_size]),name='w')#要注意的是：对于生成数据要仔细想清楚是否要加中括号
            tf.summary.histogram(layer_name+'/weights',Weights) #这一行就是完成 name命名，Weights赋值
        with tf.name_scope('biases'):
            biases = tf.Variable(tf.zeros([1,out_size])+0.1,name='b')
            tf.summary.histogram(layer_name+'/biases',biases)  #这一行也是 命名和赋值
        with tf.name_scope('Wx_plus_b'):
            Wx_plus_b = tf.add(tf.matmul(inputs,Weights),biases)
        if activation_function is None:
            outputs = Wx_plus_b
        else:
            outputs = activation_function(Wx_plus_b)
            tf.summary.histogram(layer_name+'outputs',outputs) #命名  赋值
        return outputs
    
#生成用于训练的数据
x_data = np.linspace(-1,1,300)[:,np.newaxis]
noise = np.random.normal(0,0.05,x_data.shape)
y_data = np.square(x_data)-0.5+noise


#添加用于输入，输出数据的placeholder
with tf.name_scope('inputs'):
    xs = tf.placeholder(tf.float32,[None,1],name='x_input')  #这里的[None，1]是代表不规定有多少个样本，但是规定每一个样本的大小.对于这里就是
                                                             #每次输入一个，因为只有一个特征。假如是图像的话，就是输入图像的像素个数。
    ys = tf.placeholder(tf.float32,[None,1],name='y_input')

#添加隐藏层
l1 = add_layer(xs,1,10,n_layer=1,activation_function=tf.nn.relu)  #因为定义的神经网络函数中增添了，n_layer这个参数，所以要加

#添加输出层
prediction = add_layer(l1,10,1,n_layer=2,activation_function=None) #因为定义的神经网络函数中增添了，n_layer这个参数所以要加

#设置代价函数
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.reduce_sum(tf.square(ys-prediction),reduction_indices=[1]))
    tf.summary.scalar('loss',loss)  #命名 赋值
with tf.name_scope('train'):
    train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

#神经网络最重要的一步就是初始化全局所有要用到的变量

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

#在数据可视化当中非常非常重要的步骤，
merged = tf.summary.merge_all()  #合并到summary中
writer = tf.summary.FileWriter('logs2/',sess.graph)        #⭐⭐在可视化当中，这一步是很恨重要的。 选定可视化存储目录，
                                                            #这个文件夹的名字就是  logs2

# #把生成的数据可视化

fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.scatter(x_data,y_data)
plt.ion()
plt.show()

#开始训练数据
for i in range(1000):
    sess.run(train_step,feed_dict={xs:x_data,ys:y_data})
    if i % 50 == 0:
        # merged也是需要sess.run的
        result = sess.run(merged,feed_dict={xs:x_data,ys:y_data})
        writer.add_summary(result,i)
        #然后就是添加到可视化图中
        
        print(sess.run(loss,feed_dict={xs:x_data,ys:y_data}))